# Transforming an a priori method into an interactive one
Examples fot the PPSN article

In [1]:
# Package for plotting the results
import plotly.graph_objects as go

# Package to create the Problem object
from desdeo_problem.testproblems.TestProblems import test_problem_builder

# Packages for using MOEA/D and the available scalarization functions
from desdeo_emo.EAs.MOEAD_NUMS_PLUS import MOEA_D_NUMS_PLUS
from desdeo_emo.EAs.MOEAD import MOEA_D
from desdeo_emo.EAs.Base_Interactive_apriori import BaseInteractiveApriori
from desdeo_problem import variable_builder, ScalarObjective, VectorObjective, MOProblem
from desdeo_problem.problem import Variable
import numpy as np
import pandas as pd

# create the problem
def f_1(x):
    x = np.atleast_2d(x)
    return (
        1640.2823
        + 2.3573285 * x[:, 0]
        + 2.3220035 * x[:, 1]
        + 4.5688768 * x[:, 2]
        + 7.7213633 * x[:, 3]
        + 4.4559504 * x[:, 4]
    )


def f_2(x):
    x = np.atleast_2d(x)
    return (
        6.5856
        + 1.15 * x[:, 0]
        - 1.0427 * x[:, 1]
        + 0.9738 * x[:, 2]
        + 0.8364 * x[:, 3]
        - 0.3695 * x[:, 0] * x[:, 3]
        + 0.0861 * x[:, 0] * x[:, 4]
        + 0.3628 * x[:, 1] * x[:, 3]
        - 0.1106 * x[:, 0] ** 2
        - 0.3437 * x[:, 2] ** 2
        + 0.1764 * x[:, 3] ** 2
    )

def f_3(x):
    x = np.atleast_2d(x)
    return (
        -0.0551
        + 0.0181 * x[:, 0]
        + 0.1024 * x[:, 1]
        + 0.0421 * x[:, 2]
        - 0.0073 * x[:, 0] * x[:, 1]
        + 0.024 * x[:, 1] * x[:, 2]
        - 0.0118 * x[:, 1] * x[:, 3]
        - 0.0204 * x[:, 2] * x[:, 3]
        - 0.008 * x[:, 2] * x[:, 4]
        - 0.0241 * x[:, 1] ** 2
        + 0.0109 * x[:, 3] ** 2
    )
f1 = ScalarObjective(name="f1", evaluator=f_1)
f2 = ScalarObjective(name="f2", evaluator=f_2)
f3 = ScalarObjective(name="f3", evaluator=f_3)

x_1 = Variable("x_1", 2.0, 1.0, 3.0)
x_2 = Variable("x_2", 2.0, 1.0, 3.0)
x_3 = Variable("x_3", 2.0, 1.0, 3.0)
x_4 = Variable("x_4", 2.0, 1.0, 3.0)
x_5 = Variable("x_5", 2.0, 1.0, 3.0)
variables = [x_1, x_2, x_3, x_4, x_5]
problem = MOProblem(variables=variables, objectives=[f1, f2, f3])

solutions_display = 5

def plot_complete(data, pareto_front):
    trace1 = go.Scatter3d(x=pareto_front[:,0],y=pareto_front[:,1],z=pareto_front[:,2],mode='markers',marker_size=1)
    trace2 = go.Scatter3d(x=data[:,0],y=data[:,1],z=data[:,2],mode='markers',marker_size=1)
    fig = go.Figure(data= [trace1,trace2])
    return fig




def plot_results_scatter(data):
    fig2 = go.Figure(data=go.Scatter3d(x=data[:,0],
                                   y=data[:,1],
                                   z=data[:,2],
                                   mode="markers",
                                   marker_size=1))
    return fig2



Create the object and run the initialization step

In [2]:
int_nums_plus = MOEA_D_NUMS_PLUS(problem=problem, interact=True, n_gen_per_iter=100, roi_size=0.3, num_solutions_display=5)


int_nums_plus.start()

print("Ideal point", int_nums_plus._global_ideal_point)
print("Nadir point", int_nums_plus._global_nadir_point)

front = np.copy(int_nums_plus.population.objectives)
fig = plot_results_scatter(int_nums_plus.population.objectives)
fig

Ideal point [1.66170805e+03 7.24320849e+00 7.08057506e-02]
Nadir point [1.66533280e+03 8.30453456e+00 1.06958748e-01]


Ask for preference information. Then, run the a priori algorithm

In [3]:
reference_point = [int_nums_plus._global_ideal_point[0],int_nums_plus._global_ideal_point[1],int_nums_plus._global_ideal_point[2]]

pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.requests()
pref_int_nums_plus[2].response = pd.DataFrame([reference_point], columns=pref_int_nums_plus[2].content['dimensions_data'].columns)
pref_int_nums_plus, plot_int_nums_plus  = int_nums_plus.iterate(pref_int_nums_plus[2])
print(int_nums_plus._current_display_solutions)
print(int_nums_plus._reference_point_list)

fig = plot_complete(int_nums_plus.population.objectives,front)
fig


/home/giomara/PPSN2022/desdeo-emo/desdeo_emo/utilities/ReferenceVectors.py:443: RuntimeWarning:

invalid value encountered in true_divide



[[1.66314985e+03 7.88236221e+00 9.82080053e-02]
 [1.66204375e+03 8.20623710e+00 7.88457334e-02]
 [1.66507144e+03 7.31970663e+00 1.05839963e-01]
 [1.66395799e+03 7.64573386e+00 1.05439724e-01]
 [1.66253699e+03 8.06181433e+00 8.88309615e-02]]
[[1.66170805e+03 7.24320849e+00 7.08057506e-02]]


Select reference point closer to the front

In [4]:
reference_point = [1661.9,8,1]
pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.requests()
pref_int_nums_plus[2].response = pd.DataFrame([reference_point], columns=pref_int_nums_plus[2].content['dimensions_data'].columns)
pref_int_nums_plus, plot_int_nums_plus  = int_nums_plus.iterate(pref_int_nums_plus[2])
print(int_nums_plus._current_display_solutions)
print(int_nums_plus._reference_point_list)

fig = plot_complete(int_nums_plus.population.objectives,front)
fig

[[1.66207094e+03 8.19827746e+00 7.94526825e-02]
 [1.66229183e+03 8.13359715e+00 8.41398233e-02]
 [1.66219291e+03 8.16256311e+00 8.20947017e-02]
 [1.66201869e+03 8.21357676e+00 7.82802060e-02]
 [1.66212209e+03 8.18329874e+00 8.05769475e-02]]
[[1.66170805e+03 7.24320849e+00 7.08057506e-02]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]]


Change direction

In [5]:
reference_point = [1665,7.4,0.1]
pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.requests()
pref_int_nums_plus[2].response = pd.DataFrame([reference_point], columns=pref_int_nums_plus[2].content['dimensions_data'].columns)
pref_int_nums_plus, plot_int_nums_plus  = int_nums_plus.iterate(pref_int_nums_plus[2])
print(int_nums_plus._current_display_solutions)
print(int_nums_plus._reference_point_list)

fig = plot_complete(int_nums_plus.population.objectives,front)
fig

[[1.66474323e+03 7.41581102e+00 1.06873982e-01]
 [1.66298206e+03 7.93149335e+00 9.59745284e-02]
 [1.66352931e+03 7.77125489e+00 1.02330699e-01]
 [1.66527448e+03 7.26025573e+00 1.04718174e-01]
 [1.66415174e+03 7.58900197e+00 1.06305865e-01]]
[[1.66170805e+03 7.24320849e+00 7.08057506e-02]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]
 [1.66500000e+03 7.40000000e+00 1.00000000e-01]]


reutilize previous

In [6]:
#Measure ASF with archive
def euclid_dist(t1, t2):
    return np.sqrt(((t1-t2)**2).sum(axis = 1))

distance = euclid_dist(int_nums_plus._reference_point_list[1],int_nums_plus._archive_objectives)
idx = np.argpartition(distance, int_nums_plus.num_solutions_display)[:int_nums_plus.num_solutions_display]

#print(distance)
#print(int_nums_plus._reference_point_list)
#print(idx)

idx_change = np.array(int_nums_plus._archive_indexes[idx]).astype(int)
#print(idx_change)
#print(int_nums_plus._archive_variables[idx])
#print(int_nums_plus._archive_indexes[idx])

#replace in the population for the next generation
int_nums_plus.population.individuals[idx_change] = int_nums_plus._archive_variables[idx]

Reduce spread size

In [7]:
int_nums_plus.roi_size = 0.2

reference_point = int_nums_plus._reference_point_list[1]
pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.requests()
pref_int_nums_plus[2].response = pd.DataFrame([reference_point], columns=pref_int_nums_plus[2].content['dimensions_data'].columns)
pref_int_nums_plus, plot_int_nums_plus  = int_nums_plus.iterate(pref_int_nums_plus[2])
print(int_nums_plus._current_display_solutions)
print(int_nums_plus._reference_point_list)

fig = plot_complete(int_nums_plus.population.objectives,front)
fig

[[1.66207462e+03 8.19720000e+00 7.95343354e-02]
 [1.66215584e+03 8.17341815e+00 8.13057539e-02]
 [1.66204035e+03 8.20723373e+00 7.87692718e-02]
 [1.66211105e+03 8.18653106e+00 8.03363172e-02]
 [1.66220141e+03 8.16007505e+00 8.22738039e-02]]
[[1.66170805e+03 7.24320849e+00 7.08057506e-02]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]
 [1.66500000e+03 7.40000000e+00 1.00000000e-01]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]]


In [8]:
int_nums_plus.roi_size = 0.1

reference_point = int_nums_plus._reference_point_list[1]
pref_int_nums_plus, plot_int_nums_plus = int_nums_plus.requests()
pref_int_nums_plus[2].response = pd.DataFrame([reference_point], columns=pref_int_nums_plus[2].content['dimensions_data'].columns)
pref_int_nums_plus, plot_int_nums_plus  = int_nums_plus.iterate(pref_int_nums_plus[2])
print(int_nums_plus._current_display_solutions)
print(int_nums_plus._reference_point_list)

fig = plot_complete(int_nums_plus.population.objectives,front)
fig

[[1.66206839e+03 8.19902388e+00 7.93960464e-02]
 [1.66212365e+03 8.18284183e+00 8.06108739e-02]
 [1.66208411e+03 8.19441929e+00 7.97445037e-02]
 [1.66205431e+03 8.20314599e+00 7.90822256e-02]
 [1.66210343e+03 8.18876445e+00 8.01694157e-02]]
[[1.66170805e+03 7.24320849e+00 7.08057506e-02]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]
 [1.66500000e+03 7.40000000e+00 1.00000000e-01]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]
 [1.66190000e+03 8.00000000e+00 1.00000000e+00]]


In [9]:
trace = go.Parcoords(int_nums_plus.population.objectives)

fig = go.Figure(data=[trace])
fig

ValueError: The first argument to the plotly.graph_objs.Parcoords 
constructor must be a dict or 
an instance of :class:`plotly.graph_objs.Parcoords`